In [1]:

!pip install numpy==1.25.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.25.0 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.25.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.25.0 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.1 which is incompatible.
tensorflow 2.12.0 requires num

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV
from sklearn.pipeline import make_pipeline
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20/548437596.py", line 8, in <module>
    from sklearn.linear_model import LogisticRegression
  File "/opt/conda/lib/python3.10/site-packages/sklearn/__init__.py", line 82, in <module>
    from .base import clone
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 17, in <module>
    from .utils import _IS_32BIT
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/__init__.py", line 25, in <module>
    from .fixes import parse_version, threadpool_info
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/fixes.py", line 19, in <module>
    import scipy.stats
  File "/opt/conda/lib/python3.10/site-packages/scipy/stats/__init__.py", line 441, in <module>
    from .stats import *
  File "/opt/conda/lib/python3.10/site-packages/scipy/stats/

# Read Dataset

In [ ]:
dataset_path = '/kaggle/input/cancer-data/Cancer_Data.csv'

# Data Info

In [ ]:
dataset = pd.read_csv(dataset_path)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
corel = dataset.corr()
corel

# Exploratory Data Analysis

**** Coorealation

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(corel, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

**** Distribution of the target variable

In [ ]:
sns.countplot(x='diagnosis', data=dataset)
plt.title('Diagnosis Distribution')
plt.show()

**** Pair plot for selected features

In [ ]:
selected_features = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean']
sns.pairplot(dataset[selected_features], diag_kind='kde')
plt.suptitle('Pair Plot of Selected Features')
plt.show()

**** Box plots for selected features by diagnosis

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='diagnosis', y='radius_mean', data=dataset)
plt.title('Box Plot of Radius Mean by Diagnosis')
plt.show()

**** Histograms of features

In [ ]:
dataset.drop(['Unnamed: 32'], axis=1, inplace=True)  # Dropping the empty column
dataset.hist(bins=20, figsize=(15, 12))
plt.suptitle('Histograms of Features', y=1.02)
plt.show()

# Data Featuring

In [ ]:
features = dataset.drop(['id', 'diagnosis'], axis = 1)
labels = dataset['diagnosis']

In [ ]:
np.unique(labels, return_counts = True)

# Feature Engineering

In [ ]:
from imblearn.under_sampling import AllKNN
features, labels = AllKNN(n_neighbors = 7).fit_resample(features, labels)

In [ ]:
np.unique(labels, return_counts = True)

In [ ]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
labels

In [ ]:
features = np.array(features)
labels = np.array(labels)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size = 0.2,
                                               stratify = labels, random_state = True)

In [ ]:
xtrain.shape, xtest.shape

In [ ]:
np.unique(ytest, return_counts = True)

In [ ]:
def K_fold(xtrain, ytrain, estimator):
    fold_k = StratifiedKFold(n_splits = 20).split(xtrain, ytrain)
    xtrain_f, xvalid_f,ytrain_f, yvalid_f = [], [], [], []
    for k , (train, valid) in enumerate(fold_k):
        xtrain_f.append(xtrain[train]), xvalid_f.append(xtrain[valid])
        ytrain_f.append(ytrain[train]), yvalid_f.append(ytrain[valid])
        estimator.fit(xtrain[train], ytrain[train])
        st = 'k: {}, train acc: {}, and validation acc: {}'.format(k, estimator.score(xtrain[train], ytrain[train]),
                                                           estimator.score(xtrain[valid], ytrain[valid]))
        print(st)
    return xtrain_f, xvalid_f,ytrain_f, yvalid_f

# LogisticRegression Classifier

In [ ]:
logistic_regression_pipe = make_pipeline(
    RobustScaler(),
    KernelPCA(kernel = 'rbf', n_components = 20),
    LogisticRegression(solver = 'lbfgs',C = 100, random_state = 1, multi_class = 'ovr',
                      class_weight = 'balanced', max_iter = 3000)
)

In [ ]:
param_range = [0.00001, 0.0001, 0.001, 0.01, 0.1, 10, 50, 100, 1000]
train_scores , validation_scores = validation_curve(estimator = logistic_regression_pipe, 
                                                   X = xtrain, y = ytrain, 
                                                   param_name = 'logisticregression__C',
                                                   param_range = param_range,
                                                   scoring = 'accuracy',
                                                   cv = 10)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(param_range, np.max(train_scores, axis = 1), label = 'training accuracy')
plt.plot(param_range, np.max(validation_scores, axis = 1), label = 'validation accuracy')
plt.legend()

In [ ]:
logisticregression_search = GridSearchCV(
                                        scoring = 'accuracy', 
                                        estimator = logistic_regression_pipe,
                                        param_grid = [{
                                            'logisticregression__solver': ['lbfgs', 'liblinear'],
                                            'logisticregression__C': param_range
                                        }],
                                        cv = 10)
logisticregression_search.fit(xtrain, ytrain)

In [ ]:
logisticregression_search.best_score_

In [ ]:
logisticregression_search.best_params_

In [ ]:
logistic_regression_pipe = logisticregression_search.best_estimator_

In [ ]:
xtrain_f, xvalid_f,ytrain_f, yvalid_f = K_fold(xtrain, 
                                               ytrain, 
                                               logistic_regression_pipe)

In [ ]:
index = 3
xtraining, ytraining, xvalid, yvalid = xtrain_f[index], ytrain_f[index], xvalid_f[index], yvalid_f[index]
logistic_regression_pipe.fit(xtrain, ytrain)

In [ ]:
len(xtraining), len(xvalid)

In [ ]:
logistic_regression_pipe.score(xtraining, ytraining)

In [ ]:
logistic_regression_pipe.score(xvalid, yvalid)

In [ ]:
logistic_regression_pipe.score(xtest, ytest)

In [ ]:
y_pred = logistic_regression_pipe.predict(xtest)
y_pred

In [ ]:
accuracy_score(y_pred, ytest)*100

In [ ]:
print(classification_report(y_pred, ytest))

In [ ]:
sns.heatmap(confusion_matrix(y_pred, ytest), annot = True, cmap = 'Blues')

# Support Vector Machine

In [ ]:
svm_pipeline = make_pipeline(RobustScaler(),
                            KernelPCA(n_components = 12, kernel = 'rbf'),
                            SVC(kernel = 'rbf', gamma = 5, C = 100, 
                               random_state = 1, probability=True))

In [ ]:
train_scores, validation_scores = validation_curve(estimator = svm_pipeline,
                                                  X = xtrain, y = ytrain, 
                                                  scoring = 'accuracy',
                                                  param_name = 'svc__C',
                                                  param_range = param_range,
                                                  cv = 10)

In [ ]:
plt.plot(param_range, np.max(train_scores, axis = 1), label = 'training accuracy')
plt.plot(param_range, np.max(validation_scores, axis = 1), label = 'validation accuracy')
plt.legend()

In [ ]:
from sklearn.model_selection import cross_val_score
search_for_best = GridSearchCV(estimator = svm_pipeline, 
                              param_grid = [{
                                  'svc__kernel': ['rbf', 'linear'],
                                  'svc__C': param_range,
                                  'svc__gamma': param_range
                                  
                              }],
                              scoring = 'accuracy', cv = 10)
search_for_best.fit(xtrain, ytrain)

In [ ]:
search_for_best.best_score_

In [ ]:
search_for_best.best_params_

In [ ]:
svm_pipeline = search_for_best.best_estimator_

In [ ]:
xtrain_f, xvalid_f,ytrain_f, yvalid_f = K_fold(xtrain, 
                                               ytrain, 
                                               svm_pipeline)

In [ ]:
index = 18
xtraining, ytraining, xvalid, yvalid = xtrain_f[index], ytrain_f[index], xvalid_f[index], yvalid_f[index]
svm_pipeline.fit(xtrain, ytrain)

In [ ]:
len(xtraining), len(xvalid)

In [ ]:
svm_pipeline.score(xtraining, ytraining), svm_pipeline.score(xvalid, yvalid)

In [ ]:
svm_pipeline.score(xtest, ytest)

In [ ]:
y_pred = svm_pipeline.predict(xtest)
y_pred

In [ ]:
print(classification_report(y_pred, ytest))

In [ ]:
sns.heatmap(confusion_matrix(y_pred, ytest), annot = True, cmap = 'Blues')

# KNN Classifier

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
knn_pipeline = make_pipeline(
                            SelectFromModel(estimator = RandomForestClassifier(
                            n_estimators = 1000, max_depth = 2), threshold = 0.05),
                            RobustScaler(),
                            KNeighborsClassifier(metric = 'minkowski',
                                                 n_neighbors = 3, 
                                                 p = 2))

In [ ]:
search_for_best_knn_params = GridSearchCV(
                                         scoring = 'accuracy',
                                         estimator = knn_pipeline,
                                         param_grid = [{
                                             'kneighborsclassifier__n_neighbors':[2, 3, 4, 5, 6, 7, 8],
                                             'kneighborsclassifier__p': [1, 2]
                                         }],
                                         cv = 10)
search_for_best_knn_params.fit(xtrain, ytrain)

In [ ]:
search_for_best_knn_params.best_score_

In [ ]:
search_for_best_knn_params.best_params_

In [ ]:
knn_pipeline = search_for_best_knn_params.best_estimator_

In [ ]:
xtrain_f, xvalid_f,ytrain_f, yvalid_f = K_fold(xtrain, 
                                               ytrain, 
                                               knn_pipeline)

In [ ]:
index = 16
xtraining, ytraining, xvalid, yvalid = xtrain_f[index], ytrain_f[index], xvalid_f[index], yvalid_f[index]
knn_pipeline.fit(xtrain, ytrain)

In [ ]:
knn_pipeline.score(xtraining, ytraining), knn_pipeline.score(xvalid, yvalid)

In [ ]:
knn_pipeline.score(xtest, ytest)

In [ ]:
y_pred = knn_pipeline.predict(xtest)
y_pred

In [ ]:
print(classification_report(y_pred, ytest))

In [ ]:
sns.heatmap(confusion_matrix(y_pred, ytest), annot = True, cmap = 'Blues')

# Majority Voting Classifier

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import _name_estimators
class MajorityVoting:
    def __init__(self, classifiers, weights):
        self.classifiers = classifiers
        self.weights = weights   
    def fit(self, features, labels):
        return self
    def predict_proba(self, x):
        predicts_ = np.array([classifier.predict_proba(x) for classifier in self.classifiers])
        weighted_predict_proba = np.average(predicts_, 
                                           axis = 0,
                                           weights = self.weights)
        return weighted_predict_proba
    def predict(self, x):
        return np.argmax(self.predict_proba(x), axis = 1)
    def score(self, features, labels):
        y_pred = np.argmax(self.predict_proba(features), axis = 1)
        return accuracy_score(labels, y_pred)

In [ ]:
def get_proba(estimator, data):
    y_pred = estimator.predict_proba(data)
    y_pred_indexs = np.argmax(y_pred, axis = 1)
    return [y_pred[index, value] for index, value in enumerate(y_pred_indexs)]

In [ ]:
xtrain_up = np.vstack([
    get_proba(logistic_regression_pipe, xtrain),
    get_proba(svm_pipeline, xtrain),
    get_proba(knn_pipeline, xtrain)
])
xtrain_up = np.reshape(xtrain_up, (xtrain_up.shape[1], xtrain_up.shape[0]))
xtrain_up.shape

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
boost = AdaBoostClassifier(estimator = DecisionTreeClassifier(criterion = 'gini'), 
                           random_state = 1,)
boost.fit(xtrain_up, ytrain)

In [ ]:
boost.score(xtrain_up, ytrain)

In [ ]:
features_importances = np.mean([tree.feature_importances_ for tree in boost.estimators_], axis = 0)/3
features_importances

In [ ]:
features_importances = np.mean([[0.6, 0.1, 0.1], features_importances], axis = 0)
features_importances

In [ ]:
majority_vote = MajorityVoting([
                                logistic_regression_pipe, 
                                svm_pipeline, 
                                knn_pipeline,],
                                weights = features_importances)

In [ ]:
xtrain_f, xvalid_f,ytrain_f, yvalid_f = K_fold(xtrain, 
                                               ytrain, 
                                               majority_vote)

In [ ]:
index = 16
xtraining, ytraining, xvalid, yvalid = xtrain_f[index], ytrain_f[index], xvalid_f[index], yvalid_f[index]
majority_vote.fit(xtrain, ytrain)

In [ ]:
majority_vote.score(xtraining, ytraining), majority_vote.score(xvalid, yvalid)

In [ ]:
majority_vote.score(xtest, ytest)

In [ ]:
y_pred = majority_vote.predict(xtest)
y_pred

In [ ]:
print(classification_report(y_pred, ytest))

In [ ]:
sns.heatmap(confusion_matrix(y_pred, ytest), annot = True, cmap = 'Blues')

## Final Result - For Testing Data:

In [ ]:
res = pd.DataFrame({
    'Models Used': ['LogisticRegression', 
                    'KNN Classification', 
                    'Support Vector Machine', 
                    'Majority Voting'],
    'Testing Data Accuracy': [logistic_regression_pipe.score(xtest, ytest)*100,
                             knn_pipeline.score(xtest, ytest)*100,
                             svm_pipeline.score(xtest, ytest)*100,
                             majority_vote.score(xtest, ytest)*100]
})
res.head()

## Final Result - For All Data in Dataset:

In [ ]:
res = pd.DataFrame({
    'Models Used': ['LogisticRegression', 
                    'KNN Classification', 
                    'Support Vector Machine', 
                    'Majority Voting'],
    'All Data Accuracy': [logistic_regression_pipe.score(features, labels)*100,
                             knn_pipeline.score(features, labels)*100,
                             svm_pipeline.score(features, labels)*100,
                             majority_vote.score(features, labels)*100]
})
res.head()